In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
NB_USERS = 5000
FEAT_USER = 3
NB_ITEMS = 100
FEAT_ITEM = 6

NB_EPOCHS = 500
LAMBDA_REG = 1e-5
# LAMBDA_REG = 0
# learning_rate = 0.001

In [3]:
# Load data
users = pd.read_csv('data/sushi/sushi3.udata', sep='\t', names=('uid', 'gender', 'age', 'time', 'old_prefecture', 'old_region', 'old_eastwest', 'prefecture', 'region', 'eastwest', 'same'))
items = pd.read_csv('data/sushi/sushi3.idata', sep='\t', names=('iid', 'name', 'style', 'major', 'minor', 'heaviness', 'frequency', 'price', 'popularity'))
R = pd.read_csv('data/sushi/sushi3b.5000.10.score', sep=' ', header=None)
triplets = []
for i, line in enumerate(np.array(R)):
    for j, v in enumerate(line):
        if v != -1:
            triplets.append((i, j, v))
df_ratings = pd.DataFrame(triplets, columns=('user', 'item', 'rating'))
train, test = train_test_split(df_ratings, test_size=0.2, shuffle=True)

In [4]:
# TF
#A is not used (only the "criterion on the items" => only B)
#A = tf.constant(np.array(users[['age', 'gender', 'region']]).astype(np.float32)) 
B = tf.constant(np.array(items[['heaviness', 'frequency', 'price', 'popularity', 'style', 'major']]).astype(np.float32))

weights = tf.get_variable('W', shape=[NB_USERS, FEAT_ITEM], dtype=np.float32, initializer=tf.truncated_normal_initializer(stddev=1))

alpha = tf.get_variable("alpha", shape=[FEAT_ITEM],
                            initializer=tf.truncated_normal_initializer(stddev=1))
beta = tf.get_variable("item_bias", shape=[FEAT_ITEM],
                            initializer=tf.truncated_normal_initializer(stddev=1))

user_batch = tf.placeholder(tf.int32, shape=[None])
item_batch = tf.placeholder(tf.int32, shape=[None])
rate_batch = tf.placeholder(tf.float32, shape=[None])

weight_users = tf.nn.embedding_lookup(weights, user_batch)

#beta_crit = tf.nn.embedding_lookup(item_bias, item_batch)
#alpha_crit = tf.nn.embedding_lookup(user_bias, user_batch)

feat_items = tf.nn.embedding_lookup(B, item_batch)
#feat_users = tf.nn.embedding_lookup(A, user_batch)

pred =tf.reduce_sum(tf.multiply(tf.nn.softmax(tf.multiply(feat_items, alpha)+beta),weight_users),1)

cost_l2 = tf.losses.mean_squared_error(rate_batch, pred)

regularizer=tf.nn.l2_loss(weight_users)

#l2_user = tf.nn.l2_loss(weight_users)
#l2_item = tf.nn.l2_loss(weight_items)
#l2_bias_user = tf.nn.l2_loss(bias_users)
#l2_bias_item = tf.nn.l2_loss(bias_items)
#regularizer = tf.add(l2_user, l2_item)
#regularizer = tf.add(regularizer, l2_bias_user)
#regularizer = tf.add(regularizer, l2_bias_item)
# regularizer = tf.nn.l2_loss(M)
penalty = tf.constant(LAMBDA_REG, dtype=tf.float32, shape=[])
cost = tf.add(cost_l2, tf.multiply(regularizer, penalty))

In [16]:
global_step = tf.train.get_global_step()
train_op = tf.train.AdamOptimizer(0.1).minimize(cost, global_step=global_step)

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(NB_EPOCHS):
        _, train_pred, train_mse, reg, pen, train_cost = sess.run([train_op, pred, cost_l2, regularizer, penalty, cost], feed_dict={
            user_batch: train['user'],
            item_batch: train['item'],
            rate_batch: train['rating']
        })
        test_pred, test_mse = sess.run([pred, cost_l2], feed_dict={
            user_batch: test['user'],
            item_batch: test['item'],
            rate_batch: test['rating']
        })
        print('train rmse', train_mse ** 0.5, 'test rmse', test_mse ** 0.5)
        # print('reg', reg, 'full cost', train_cost)
        
    #print(weights.eval(session=sess))
    W_mat=weights.eval(session=sess)
    alpha_vec= alpha.eval(session=sess)
    beta_vec= beta.eval(session=sess)
    

train rmse 3.043606457468563 test rmse 2.9442904290119025
train rmse 2.955217377078812 test rmse 2.861597583568599
train rmse 2.8711359345327816 test rmse 2.7804012932436475
train rmse 2.7890179627866916 test rmse 2.6997193508519097
train rmse 2.707494830886542 test rmse 2.619899062476396
train rmse 2.626626736810232 test rmse 2.541334526581432
train rmse 2.546658653010331 test rmse 2.4641378271457657
train rmse 2.4677236813902184 test rmse 2.3884672512391076
train rmse 2.390025543650988 test rmse 2.3146385408324623
train rmse 2.313867525467921 test rmse 2.2430533089903717
train rmse 2.239624366088648 test rmse 2.1740809153963596
train rmse 2.167665642521594 test rmse 2.10797219431452
train rmse 2.098283670315933 test rmse 2.0449247897618315
train rmse 2.0317203857396513 test rmse 1.9851842116391203
train rmse 1.9682487576322196 test rmse 1.9289688440103743
train rmse 1.9080874233250606 test rmse 1.8763489003708376
train rmse 1.8512896888079393 test rmse 1.8272008271609637
train rmse 1

train rmse 1.185174924518511 test rmse 1.2984447401007861
train rmse 1.1851574731165408 test rmse 1.2984311522547858
train rmse 1.1851392670573222 test rmse 1.298417380644154
train rmse 1.185121312189093 test rmse 1.2984049860696958
train rmse 1.1851040108822481 test rmse 1.2983926372834058
train rmse 1.185087866121009 test rmse 1.298380885169538
train rmse 1.1850719726209662 test rmse 1.2983690870419775
train rmse 1.1850573363295502 test rmse 1.2983565542832103
train rmse 1.185042297477568 test rmse 1.298345398648915
train rmse 1.1850286667812913 test rmse 1.298334105193105
train rmse 1.1850157401089785 test rmse 1.2983234543642832
train rmse 1.1850031150905693 test rmse 1.2983133543596876
train rmse 1.1849914456312236 test rmse 1.2983041724691458
train rmse 1.1849800778577413 test rmse 1.2982959087121373
train rmse 1.1849688105764131 test rmse 1.2982883794655131
train rmse 1.1849584989080932 test rmse 1.2982817683836945
train rmse 1.1849486398632514 test rmse 1.2982758918382602
train

train rmse 1.1846086035166619 test rmse 1.2982304854458875
train rmse 1.1846091066755275 test rmse 1.298232872879177
train rmse 1.184609911729268 test rmse 1.298235260308076
train rmse 1.1846104652033973 test rmse 1.2982373722607499
train rmse 1.1846115218351077 test rmse 1.298239713769481
train rmse 1.1846123772029449 test rmse 1.2982419175385467
train rmse 1.1846133332015614 test rmse 1.298244167215609
train rmse 1.1846137860427421 test rmse 1.2982465546237383
train rmse 1.1846147420402218 test rmse 1.2982488961159082
train rmse 1.1846159999304668 test rmse 1.2982513753383087
train rmse 1.1846165534017514 test rmse 1.2982533954419493
train rmse 1.1846177106590567 test rmse 1.2982560583010365
train rmse 1.1846182138140542 test rmse 1.2982581702198772
train rmse 1.1846193710697372 test rmse 1.2982599607570728
train rmse 1.1846203773781079 test rmse 1.2982623481361588
train rmse 1.1846216855777119 test rmse 1.2982647814218635
train rmse 1.1846222390463401 test rmse 1.2982672606139287
tr

train rmse 1.184784042059294 test rmse 1.2985522443288626
train rmse 1.1847854003865752 test rmse 1.2985544016667667
train rmse 1.1847866077872953 test rmse 1.298555824589806
train rmse 1.1847879158033536 test rmse 1.2985574770146044
train rmse 1.1847890225850444 test rmse 1.2985597261449788
train rmse 1.1847901796738882 test rmse 1.2985614703658528
train rmse 1.1847912361453241 test rmse 1.298563076883006
train rmse 1.1847922926158179 test rmse 1.2985653260036816
train rmse 1.1847936509336399 test rmse 1.2985671161173535
train rmse 1.1847948080179636 test rmse 1.2985690898295674
train rmse 1.1847961663329019 test rmse 1.2985705586367098
train rmse 1.1847973737226507 test rmse 1.2985724405434331
train rmse 1.1847986314189973 test rmse 1.2985741847472292
train rmse 1.1847997884984571 test rmse 1.2985755617485715
train rmse 1.1848008449613248 test rmse 1.2985772600482157
train rmse 1.1848018511155611 test rmse 1.2985792337450117
train rmse 1.1848032597300566 test rmse 1.2985811615389864


In [17]:
print(W_mat)
print(alpha_vec)
print(beta_vec)

[[ 2.6750633e-05  1.1477541e+00  2.8907898e-06  4.8404086e-06
   3.4291856e-02  9.7424993e-07]
 [-6.1952232e-07  5.9558117e-01 -1.5323755e-08 -2.1215232e-08
  -8.9905027e-04 -2.8798068e-08]
 [-5.5757242e-05  2.1133094e+00 -7.3574924e-06 -7.6904516e-06
  -5.5326227e-02 -8.7951577e-07]
 ...
 [ 1.3119827e-06  1.9459037e+00 -2.2421301e-07 -8.5221046e-08
   2.3196761e-03  7.1165225e-08]
 [ 5.5709603e-07  2.5175779e+00  1.7284140e-07 -2.4585785e-07
  -2.9398920e-04  2.3925807e-07]
 [-4.6793176e-07  2.6858563e+00 -1.0248291e-07 -5.4735527e-10
  -4.9463171e-04 -1.6811669e-08]]
[ 0.0399156   7.96495    -0.78659195 -2.746602    2.6555805  -1.2326641 ]
[-0.69059277  3.6026833  -1.6191064  -2.2190504   3.6800885  -3.942153  ]


In [15]:
import matplotlib.pyplot as plt
plt.imshow(tmp)

ModuleNotFoundError: No module named 'matplotlib'